# Bases de connaissance : interroger Wikidata à l'aide de requêtes SPARQL

## Imports

In [4]:
from datetime import datetime as dt # importer la librairie qui permet de comparer les requêtes à la date du jour
from SPARQLWrapper import SPARQLWrapper, JSON # importer la librairie sparql

## Obtenir la liste des politiciens belges

In [5]:
# Retrieve results from SPARQL
endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql" # wikidata au format RDF
sparql = SPARQLWrapper(endpoint) # stockage de l'adressage dans une variable

# P27 = country of citizenship 
# Q31 = Belgium
# P106 = Opccupation
# Q82955 = Politician
# P569 = date of birth
# P570 = date of death

# stocker la requête dans une variable
# quels sont les uri's des politiciens belges, leur label/nom, leurs dates de naissance et de mort (s'ils sont morts) ?

statement = """
SELECT DISTINCT ?person ?personLabel ?dateBirth ?dateDeath WHERE {
    ?person wdt:P27 wd:Q31 .
    ?person wdt:P106 wd:Q82955 .
    ?person wdt:P569 ?dateBirth .
    OPTIONAL {?person wdt:P570 ?dateDeath .}
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
ORDER BY ?personLabel
"""

sparql.setQuery(statement) # adresse la requête à wikidata
sparql.setReturnFormat(JSON) # retourner le résultat au format json
results = sparql.query().convert() # stocker les résultats convertis dans une variable

rows = results['results']['bindings'] # créer une liste de résultats avec le lien associé à chaque label
print(f"\n{len(rows)} Belgian politicians found\n") # imprimer le nombre de politiciens recensés par la requête
print(rows[:10]) # imprimer les 10 premiers résultats


7657 Belgian politicians found

[{'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q17350686'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Abdallah Kanfaoui'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1968-05-21T00:00:00Z'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q13461801'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Abdelaziz Charkaoui Hajri'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1951-01-01T00:00:00Z'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q27903935'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Abderrahim Lahlali'}, 'dateBirth': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime', 'type': 'literal', 'value': '1978-01-01T00:00:00Z'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3294738'}, 'person

## Filtrer pour n'afficher que les noms contenant "Bouchez"

In [31]:
name_filter = 'Bouchez' # créer une variable pour le filtre
max_results = 10 # limiter le résultat à 10 occurences

date_format = "%Y-%m-%dT%H:%M:%SZ" # définir le format à appliquer aux dates
filtered_rows = [row for row in rows if name_filter in row['personLabel']['value']] # appliquer le filtre et s'il existe, prendre la valeur

print(f"Displaying the first {max_results}:\n") # imprimer la phrase d'introduction des résultats (limités à 10 ici)

for row in filtered_rows[:max_results]: # pour que chaque ligne des 10 premiers résultats
    try:
        birth_date = dt.strptime(row['dateBirth']['value'], date_format) # stocker la valeur de la date de naissance au format date (string au départ)
        birth_year = birth_date.year # ne garder que l'année de la date de naissance
    except ValueError:
        birth_year = "????" # ignorer les valeurs inconnues
    try:
        death_date = dt.strptime(row['dateDeath']['value'], date_format) # idem pour la date de mort
        death_year = death_date.year
    except ValueError: # unknown death date
        death_date = "????"
    except KeyError: # still alive
        death_date = "" # ignorer l'erreur si l'année de mort n'est pas encore connue
    print(f"{row['personLabel']['value']} ({birth_date}-{death_date})") # afficher la date complète en choississant la bonne variable

Displaying the first 10:

Georges-Louis Bouchez (1986-03-23 00:00:00-)


In [26]:
help(dt.strptime)

Help on built-in function strptime:

strptime(...) method of builtins.type instance
    string, format -> new datetime parsed from a string (like time.strptime()).



## Nouvelle requête pour les bulletins de la Ville de Bruxelles

In [45]:
endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql" # wikidata au format RDF
sparql = SPARQLWrapper(endpoint) # stockage de l'adressage dans une variable

# P39 = fonction occupée par une personne
# Q33126365 = bourgmestre de Bruxelles
# P580 = date de début
# P582 = date de fin

# stocker la requête dans une variable
# quels sont les uri's des politiciens belges, leur label/nom, leurs dates de naissance et de mort (s'ils sont morts) ?

statement = """
SELECT DISTINCT ?person ?personLabel ?timeStart ?endDate WHERE {
    ?person wdt:P39 wd:Q33126365 .
    OPTIONAL {?person wdt:P580 ?timeStart .}
    OPTIONAL {?person wdt:P582 ?endDate .}
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
ORDER BY ?personLabel
"""

sparql.setQuery(statement) # adresse la requête à wikidata
sparql.setReturnFormat(JSON) # retourner le résultat au format json
results = sparql.query().convert() # stocker les résultats convertis dans une variable

rows = results['results']['bindings'] # créer une liste de résultats avec le lien associé à chaque label
print(f"\n{len(rows)} Bourgmestres de la Ville de Bruxelles trouvés\n") # imprimer le nombre de politiciens recensés par la requête
print(rows[:10]) # imprimer les 10 premiers résultats


46 Bourgmestres de la Ville de Bruxelles trouvés

[{'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2535128'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Adolphe Max'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q106764198'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': "Adrien de Grimberghe, seigneur d'Assche"}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2744763'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'André-Napoléon Fontainas'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q106764214'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Antoine de Grimbergen van Assche'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q716382'}, 'personLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Charles Buls'}}, {'person': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1066539'}, 'person

## Pour en savoir plus

- Le projet Wikidata : https://www.wikidata.org/wiki/Wikidata:Main_Page
- Aide à la construction de requêtes : https://query.wikidata.org/